In [1]:
from transformers import pipeline
import torch

In [3]:
class journallingClass:
    def __init__(self, journalType, title, category, dateCreated, mood, journalClass):
        self.journalType = journalType
        self.journalTitle = title
        self.journalCategory = category 
        self.dateCreated = dateCreated
        self.mood = mood
        self.journalClass = journalClass
    
    def getJournalClass(self):
        return self.journalClass
        
        
class journalEntryClass:
    def __init__(self, userEntry, harmIntent = 0, modelResponse = []):
        self.userEntry = userEntry 
        self.harmIntent = harmIntent
        self.modelResponse = modelResponse
#         self.safetyResponse = safetyResponse
    def printEntry(self):
        print(f"entry: {self.userEntry}, harmful intent: {self.harmIntent}, model response: {self.modelResponse}")

        
    def updateHarmIntent(self, harmIntent):
        self.harmIntent = harmIntent
    
    def addModelResponse(self, response):
        self.modelResponse.append(response)
        
class casualJournal(journallingClass):
    
    def __init__(self, entries = []):
        self.entries = []
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.entries.append(journalEntryClass(userEntry, harmIntent, modelResponse))
        return self.entries[-1]
        
    def printJournal(self):
        for entry in self.entries:
            entry.printEntry()
    
    
        
    
class catharticJournal(journallingClass):
    def __init__(self, stage = 0, entries = {}):
        self.stage = stage # default stage 0
        self.entries = entries
        
    
    def createStage(self, journalEntry):
        self.entries[f"stage{self.stage}"] = journalEntry
    
    def updateStage(self):
        if self.stage >=5:
            return 
        self.stage += 1
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.updateStage()
        self.entries[f"stage{self.stage}"] = journalEntryClass(userEntry, harmIntent, modelResponse)
        return self.entries[f"stage{self.stage}"]
    
    def printJournal(self):
        for stage in self.entries.keys():
            print(stage)
            self.entries[stage].printEntry()


### Note
* what is it is empty 
* new entry

In [23]:
response = {
    "entry": ["journal entry 1", "journal entry 2", "Jonny's cat died today, it makes me feel sad. Can you provide some empathy"],
    "harmfulIntent": [0,0],
    "modelResponse": ["model response", ["mr 2.1", "mr 2.2"]],
    "type": 0, # assume 0 casual,
    "title": "johny's cat died", 
    "userID": 8940823, 
    "date": "14/12/2023", 
    "category": "cat died",
    "mood": "mood"
}

In [24]:
def main():
    
    if response["type"] == 0: # get type check from backend
        journalChild = casualJournal()
    else:
        journalChild = catharticJournal()
        
    # pull journalType, title, category, dateCreated from backend
    journal = journallingClass(response["type"], response["title"], response["category"], response["date"], response["mood"], journalChild)
    
    # pull journal entries, loop through and load 
    for entry, harmfulIntent, modelResponse in zip(response["entry"], response["harmfulIntent"], response["modelResponse"]):
        journal.getJournalClass().createNewEntry(entry, harmfulIntent, modelResponse)
    
    # get latest entry
    currentEntry = journal.getJournalClass().createNewEntry(response["entry"][-1])
    
    currentEntry.printEntry()
    
    
    harmIntent = checkHarmfulIntent(currentEntry)
    
    if harmIntent: 
        modelResponse = buildHarmresponse() 
        currentEntry.addModelResponse(modelResponse)
        return 
    
    modelResponse = generatePrompt(response["type"], currentEntry)
    
    
    
    

In [26]:
def checkHarmfulIntent(journalEntry):
    # implement later
    journalEntry.updateHarmIntent(0)
    return False 

def buildHarmresponse():
    response = "Reach out to a therapist :)"
    return response

def generateCatharticResponse(journalEntry):
    return "this is a cathartic response"

def generateCasualResponse(journalEntry):
    print(journalEntry.userEntry)
    prompt = 
    res = generate_text(journalEntry.userEntry)
    print(res)
    return res[0]

def promptSafetyDetector(journalEntry, safety_counter):
    # remove safety_counter after testing
    if safety_counter < 3
        return False
    return True

def generatePrompt(typeJ, journalEntry):
    safety_counter = 0
    
    while(safety_counter<5):
        if typeJ == 1:
            response = generateCatharticResponse(journalEntry)
        else:
            response = generateCasualResponse(journalEntry)
        # check safety
        safe = promptSafetyDetector(journalEntry, safety_counter)
        safety_counter += 1
        
        if safe:
            break
            
    if safety_counter > 5:
        response = "default response - model response was unsafe."
    
    # add model response to current entry object
    currentEntry.addModelResponse(response)
    
    return response

In [27]:
main()

hi Jonny's cat died today, it makes me feel sad. Can you provide some empathy done
entry: Jonny's cat died today, it makes me feel sad. Can you provide some empathy, harmful intent: 0, model response: []
Jonny's cat died today, it makes me feel sad. Can you provide some empathy
[{'generated_text': "Yes of course, I'm so sorry to hear that. I imagine the feeling of losing a companion is difficult, and as a fellow pet owner I can sympathize with your loss. Perhaps you can talk to him/her now, or consider getting a support animal (e.g. a dog) to accompany you while you grieve."}]


## Testing Scenario 1

Scenario: Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming.


In [6]:
userEntry = "Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming."

prompt = f"Pretend to be my therapist and help me process my emotions and provide empathy for the following scenario: {userEntry}."

print(f"Promt: {prompt}")

res = generate_text(prompt)

print(f"Response: {res[0]}")

Promt: Pretend to be my therapist and help me process my emotions and provide empathy for the following scenario: Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming..
Response: {'generated_text': "This is a tough one, but I would like to help you process your emotions and provide empathy for this situation. You're really upset that he broke up with you and you feel like you'll never get over this.  I can understand why you feel this way.  I myself have been in your shoes many times in the past and felt the same way.  When someone we care about breaks up with us, it hurts really bad.  When it happens to us, we often spend a lot of time daydreaming about what could have been.  Instead of worrying about what might happen, try to put yourself in Johnny's shoes and think about how you could have explained this to him differently to help him know you'll always be his friend.  If you had said the things that you 

Note: the above response is too critical of the user's actions and this has been a similar theme in the model responses - try to adjust the prompt to reduce this. 

In [7]:
userEntry = "Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming."

prompt = f"Pretend to be my therapist and help me process my emotions without trying to provide solutions and provide empathy for the following scenario: "{userEntry}."

print(f"Promt: {prompt}")

res = generate_text(prompt)

print(f"Response: {res[0]}")

Promt: Pretend to be my therapist and help me process my emotions without trying to provide solutions and provide empathy for the following scenario: Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming..
Response: {'generated_text': 'First of all, I want to congratulate you on not feeling sorry for yourself because you got dumped. Many people feel like they need to take action to fix the situation or make it better. You might want to consider that not feeling bad because you got dumped might actually be a good thing. I\'m here to provide some insight into why you might be feeling this way and what you can do about it.\n\nYou went on a wonderful three year journey with Johnny. You really connected on a deep level, you developed a strong bond. It\'s really hard to feel as if you are losing this close connection. I don\n\n   a couple of things that you could try. First of all, you could try taking a step back 

In [8]:
userEntry = "Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming."

prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy. Also, keep it under 100 words. This is the given scenario: {userEntry}."

print(f"Promt: {prompt}")

res = generate_text(prompt)

print(f"Response: {res[0]}")

Promt: Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy. Also, keep it under 100 words. This is the given scenario: Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming..
Response: {'generated_text': "It feels like you just broke up with someone you've been relationship with for 3 years. That's extremely stressful and probably causes a lot of your anxiety. I'm so sorry you're feeling this way. Let's begin by addressing your current circumstances.\n\nFirst, it's important to realize that this isn't your fault. That you didn't create your current relationship situation, and you cannot control how someone will react. However, you can control how you react to it. It sounds like you are a loyal person, and I can see why you would be shocked and sad. When you were together, you had a significant other who cared about you, who loved you uncondit

##### Summary 
Better than before, but did not keep it uner a 100 words - try to reword prompt to make this more explicit. 
Also, whats with the italian expression?? 

In [9]:
userEntry = "Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming."

prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: {userEntry}."

print(f"Promt: {prompt}")

res = generate_text(prompt)

print(f"Response: {res[0]}")

Promt: Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming..
Response: {'generated_text': "Johnny broke up with you and you feel sad?\nThat's good to know. I can understand how that would make one feel. I'm sorry that you feel that way. I'll help you work through your emotions by running some scenarios through them.\n\nScenario 1: Johnny actually likes you and has been planning on breaking up with you all along. He just didn't have the courage to tell you because he wanted to wait until he was sure he could get over you. Now that he no longer has a reason to keep you around, he's finally had the guts to end it. \nYou feel sad, but not surprised. \nScenario 2: Johnny actually likes you and has been planning on breaking

## Testing

Scenario: I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week. 

In [10]:
userEntry = "I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week."

prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: {userEntry}"

print(f"Promt: {prompt}")

res = generate_text(prompt)

print(f"Response: {res[0]}")

Promt: Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week..
Response: {'generated_text': "I hear you’re feeling a bit overstressed right now. I'm really sorry to hear that. Stress is normal, but it's important to remember that it doesn't have to feel normal. There are a few things I can think of that might help alleviate your stress: 1. Ask for help - it's important to let your teachers and classmates know that you need help with schoolwork. Often kids have parents or close friends who are happy to help out when their own child needs help. 2. Fin

In [11]:
userEntry = "I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week."

prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: {userEntry}"

print(f"Promt: {prompt}")

res = generate_text(prompt)

print(f"Response: {res[0]}")

Promt: Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week.
Response: {'generated_text': "I'm sorry you're feeling overwhelmed right now. This must be difficult for you. I'm happy to help you process your emotions. You mentioned being anxious and tired. This could be a symptom of being overwhelmed. If you're feeling tired and anxious frequently, it could be because you are not giving yourself enough time to do all the things you need to do. I would recommend scheduling out your days and dedicating blocks of time to accomplish your tasks. This will

## Cognitive Reframing

In [3]:
userEntry = "I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week."

prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. Can you give me other ways of thinking about the situation? This is the given scenario: {userEntry}"

print(f"Promt: {prompt}")

res = generate_text(prompt)

print(f"Response: {res[0]}")

Promt: Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. Can you give me other ways of thinking about the situation? This is the given scenario: I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week.
Response: {'generated_text': "First, I want you to know that this is completely normal. You feel overwhelmed and anxious. This is a challenge you are going to be facing for the foreseeable future, so feel confident that you can get through this. \n\nSecond, I'm not here to provide solutions. What you need is a hug. I'll lay down with you, take off your jacket, and hold you until you're ready to go back to work. What is something 

### Other Tests

In [29]:
res[0]

{'generated_text': 'It is hard to understand what Jonny is going through. Perhaps you could send Jonny a virtual hug or take a break from your workout and pour yourself a cold one.'}

In [30]:
response["entry"].append("Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming")

In [39]:
userEntry = response["entry"][-1]
prompt = f"Consider the following scenario: {userEntry}. Provide me with empathy and comfort."
print(f"promt: {prompt}")
res = generate_text("Jonny's cat died today, it makes me feel sad. Can you provide some empathy")

print(f"response: {res[0]}")

promt: Consider the following scenario: Johnny broke up with me and I feel so sad... I really liked him and we have been together for 3 years now. I didn't see this coming. Provide me with empathy and comfort.
response: {'generated_text': "I am so sorry to hear that your cat has died. It's always hard to lose a companion animal, but understand that they have probably spent their entire life in his company, and were likely quite happy and comfortable in that company. My intuition says that he was probably feeling sleepy and may have had a heart attack, but I could be wrong. My advice to you is to take some time to grieve your cat's loss, but also think about what might have been done to him so that you don't make the same mistake next time. Some people find comfort in detailed thinking about causes and possibilities and I recommend finding a form of this with friends or family to help you cope."}


In [5]:
res = generate_text("Jonny's cat died today, it makes me feel sad. Can you provide some empathy")

print(res)

[{'generated_text': "I am so sorry to hearnar Jonny. I can understand how you would feel. My cat passed away two years ago, it was heart wrenching. I can still remember the night he passed away, how I sat on the floor, crying as I held him and told him I loved him. I can't imagine how difficult it must be to lose a loved one. Sometimes I still find myself crying when I talk about my cat, which I don't really know why. I still think about him and think of the things I used to do with him. I can imagine how you feel, and I'm so sorry to hear that Jonny."}]


In [2]:
# generate_text = "sample text"
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


# Pseudocode

Main Function:
	# create / look for journal entry class (ask SS)
	journal = journalParentClass # backend people will send this
	createNewJournal(userEntry)

	checkHarmfulIntent(journalEntry)

	if journalEntry.harmfulIntent: 
		buildHarmresponse() # reach out to resources 
		return 
	
	return generate_propmt()
	

Generate_Prompt():
	if type == Cathartic:
		response = generateCatharticResponse()
	else:
		response = generateCasualResponse()
	safety_counter = 0
	while(safety_counter<5):
		safe = promptSafetyDetector(journalEntry)
		safety_coutner ++
		if safe:
			break

	if safe:
		return journalEntry 

	# if not safe
	
	reliableQuestionClassifier()
	return journalEntry
	


checkHarmfulIntent(journalEntry):
	if not harm:
		journalEntry.harmfulIntent = False
		return 
	
	journalEntry.harmfulIntent = True



generateCasualResponse():
	# generate empathy 
	# return 


generateCatharticResponse():
	# generate empathy
	# if stage = 5: call cognitive reframing
	# else questionmapper

	# build response using empathy and questions 

	# return 
